In [5]:
from trip_kinematics.HomogenTransformationMatrix import TransformationMatrix
from trip_kinematics.KinematicGroup import KinematicGroup, Transformation
import sympy as sp
from scipy.optimize import minimize

In [2]:
A = Transformation(name='gimbal_joint',values={'rx': 0, 'ry': 0, 'rz': 0}, state_variables=['rx', 'ry', 'rz'])
print(A.state)
A.set_state({"rx" :1})
print(A.state)

{'rx': 0, 'ry': 0, 'rz': 0}
{'rx': 1, 'ry': 0, 'rz': 0}


In [25]:
state=A.get_symbolic_state()
A.set_state(state)
Trafo=A.get_transformation_matrix()

# create function of from symbolic representation
orientation = sp.lambdify([state.values()],Trafo.get_rotation()[0][0])
print(orientation)
x_0 = [0,0,0]
#orientation(x_0)
solution = minimize(orientation,x_0)

print(solution)

<function _lambdifygenerated at 0x0000023FE3968280>
      fun: 1.0
 hess_inv: array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])
      jac: array([ 0.0000000e+00, -7.4505806e-09, -7.4505806e-09])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([0., 0., 0.])


$$\mathtt{\text{[[cos(ry\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\})
  -sin(rz\_\{gimbal\_joint\})*cos(ry\_\{gimbal\_joint\}) sin(ry\_\{gimbal\_joint\}) 0]
 [sin(rx\_\{gimbal\_joint\})*sin(ry\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\}) + sin(rz\_\{gimbal\_joint\})*cos(rx\_\{gimbal\_joint\})
  -sin(rx\_\{gimbal\_joint\})*sin(ry\_\{gimbal\_joint\})*sin(rz\_\{gimbal\_joint\}) + cos(rx\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\})
  -sin(rx\_\{gimbal\_joint\})*cos(ry\_\{gimbal\_joint\}) 0]
 [sin(rx\_\{gimbal\_joint\})*sin(rz\_\{gimbal\_joint\}) - sin(ry\_\{gimbal\_joint\})*cos(rx\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\})
  sin(rx\_\{gimbal\_joint\})*cos(rz\_\{gimbal\_joint\}) + sin(ry\_\{gimbal\_joint\})*sin(rz\_\{gimbal\_joint\})*cos(rx\_\{gimbal\_joint\})
  cos(rx\_\{gimbal\_joint\})*cos(ry\_\{gimbal\_joint\}) 0]
 [0.0 0.0 0.0 1.0]]}}$$

### Idee erlaube transformation selbst symbolische representation zu returnen!

In [ ]:
def get_symbolic_rep(transformation):
    return 0